In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np
import math, random


In [40]:
data = pd.read_csv('full_data.csv',delimiter=',',header=0,index_col=0)
net_input_dict = dict({
    'apparentTemperature':data['apparentTemperature'],
    'dewPoint':data['dewPoint'],
    'humidity':data['humidity'],
    'precipAccumulation':data['precipAccumulation'],
    'pressure':data['pressure'],
    'temperature':data['temperature'],
    'windBearing':data['windBearing'],
    'windSpeed':data['windSpeed'],
    'MWh':data['MWh']
    })

net_inputs = pd.DataFrame(net_input_dict)
net_inputs.head()

,MWh,apparentTemperature,dewPoint,humidity,precipAccumulation,pressure,temperature,windBearing,windSpeed
0,12151.77,24.66,32.18,0.94,0.133,1006.91,33.81,52.0,12.97
1,11668.22,24.71,32.18,0.94,0.409,1005.88,33.78,51.0,12.75
2,11448.49,23.50,31.53,0.93,0.172,1003.41,33.23,42.0,14.06
3,11434.57,24.24,32.26,0.94,0.416,1001.99,33.86,34.0,14.27
4,11681.97,23.51,31.44,0.93,0.627,1000.42,33.13,34.0,13.71


In [ ]:
##Define the model
class SimpleRNN(nn.Module):
    def __init__(self, hidden_size,num_features): ##Defines the class
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_features = num_features
        
        self.inp = nn.Linear(num_features, hidden_size) ##Input layer: fully-connected layer, takes in feature #, outputs hidden size
        
        ##Use 1 LSTM layer
        self.rnn = nn.LSTM(hidden_size, hidden_size, 1, dropout=0.05) 
        self.out = nn.Linear(hidden_size, 1) ##Output Layer: fully-connected layer, takes in hidden size, outputs size 1

    def step(self, input, hidden=None):
        
        #unsqueeze method adds a dimension of size 1
        #Linear layer
        #Input size: (N,1,num_features)
        #Outut size: (N,1,hidden_size)
        input = self.inp(input.view(1, -1)).unsqueeze(1) ##Applies the data to the input layer, returns input variable
        
        #LSTM Layer
        #Input size: (seq_length,batch,input_size)
        #Output size: (seq_length,batch,hidden_size)
        output, hidden = self.rnn(input, hidden) ##Applies the input variable to the rnn layer(s), returns output and hidden states
        
        #Linear Layer
        #Input size: (N,1,hidden_size)
        #Output size:(N,1,1)
        output = self.out(output.squeeze(1)) ##Applies the LSTM output to the output layer, returns output
        return output, hidden

    
    def forward(self, inputs, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(inputs) ##Sets the # of steps to the # of inputs
        outputs = Variable(torch.zeros(steps, 1, 1)) ##Initializes the output tensor (row vector of # of steps)
        
        for i in range(steps): ##Iterates through the sequence
            #"Teacher forcing" if force==False, uses last output from network as input to network
            #If true, uses target as the input
            if force or i == 0:
                input = inputs[i]
            else:
                input = output
                
            output, hidden = self.step(input, hidden)
            outputs[i] = output #Records output
            
        return outputs, hidden



In [ ]:
n_epochs = 1
hidden_size = 10
num_features=9

model = SimpleRNN(hidden_size,num_features) #Creates the model defined above
criterion = nn.MSELoss() #Sets loss function to MSE
optimizer = optim.SGD(model.parameters(), lr=0.01) 

losses = np.zeros(n_epochs) # #Initializes loss variable, for plotting



In [ ]:
for epoch in range(n_epochs):
    
    #We have 1 long sequence as opposed to a bunch of smaller sequences, so inner loop isn't necessary
    
    _inputs = sample(50) ##uses a sample dataset
    inputs = Variable(torch.from_numpy(_inputs[:-1]).float()) ##Prepares the input sequence
    targets = Variable(torch.from_numpy(_inputs[1:]).float()) ##Prepares the target sequence

    # Use teacher forcing 50% of the time
    force = random.random() < 0.5
    outputs, hidden = model(inputs, None, force)

    optimizer.zero_grad() ##Clears the gradients
    loss = criterion(outputs, targets) ##Calculates loss
    loss.backward() ##Performs backpropagation
    optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch > 0:
        print(epoch, loss.data[0])

    # Use some plotting library
    # if epoch % 10 == 0:
        # show_plot('inputs', _inputs, True)
        # show_plot('outputs', outputs.data.view(-1), True)
        # show_plot('losses', losses[:epoch] / n_iters)

        # Generate a test
        # outputs, hidden = model(inputs, False, 50)
        # show_plot('generated', outputs.data.view(-1), True)



In [ ]:
# Online training
hidden = None

while True:
    inputs = get_latest_sample()
    outputs, hidden = model(inputs, hidden)

    optimizer.zero_grad()
    loss = criterion(outputs, inputs)
    loss.backward()
    optimizer.step()